# Game Manager

## Setup

In [2]:
%pip install -r requirements.txt
%pip install ipywidgets
%pip install ipyfilechooser

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Management GUI

In [1]:
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

from dataclasses import dataclass, field, asdict
import datetime

from src.ScraperRepository import ScraperRepository
from src.GameScraper import GameScraper
from src.Game import Game
from src.GameList import GameList

from src.scrapers.AllTheFallenGameScraper import AllTheFallenGameScraper
# from src.scrapers.DikgamesGameScraper import DikgamesGameScraper
from src.scrapers.F95zoneGameScraper import F95zoneGameScraper
from src.scrapers.FapNationGameScraper import FapNationGameScraper
from src.scrapers.LewdCornerGameScraper import LewdCornerGameScraper
from src.scrapers.RoriwalrusGameScraper import RoriwalrusGameScraper


repository = ScraperRepository()
repository.add(AllTheFallenGameScraper)
# repository.add(DikgamesGameScraper)
repository.add(F95zoneGameScraper)
repository.add(FapNationGameScraper)
repository.add(LewdCornerGameScraper)
repository.add(RoriwalrusGameScraper)

gamelist = GameList(repository=repository)
gamelist.load()

url_input = widgets.Text(description="Url:")
add_from_url_button = widgets.Button(description="Add / Update game from Url")
def add_from_url(b):
    url = url_input.value
    if url:
        gamelist.add_game_from_url(url)
    print("DONE")
add_from_url_button.on_click(add_from_url)

save_button = widgets.Button(description="Save gamelist")
def save_changes(b):
    gamelist.save()
    gamelist.create_index()
save_button.on_click(save_changes)

filename_input = FileChooser(
    './',  # Default directory
    filename='urls.txt',  # Optional: Pre-fill filename
    select_default=False,  # Start without pre-selected file
    title='Select a fext file with URLs to add',  # Title displayed above the widget
)
add_from_file_button = widgets.Button(description="Start import")
def import_from_file(b):
    selected_file = filename_input.selected
    if selected_file:
        with open(selected_file, 'r', encoding='utf-8') as file:
            for line in file:
                clean_line = line.strip()
                # Ignore empty lines and comment lines
                if clean_line and not clean_line.startswith("#"):
                    print(f"Importing {clean_line}")
                    try:
                        gamelist.add_game_from_url(clean_line)
                    except Exception as e:
                        print(f"Error importing '{clean_line}': {e}")
    print("DONE")
add_from_file_button.on_click(import_from_file)


update_all_button = widgets.Button(description="Update all")
# TODO
# Add dropdown to limit to a source
# Checkbox to include completed ones as well
def update_all(b):
    print(f"Checking for updates, {datetime.datetime.now()}")
    updates = gamelist.update_all()
    print("DONE")
update_all_button.on_click(update_all)

add_from_url_ui = widgets.HBox([url_input, add_from_url_button])
add_from_url_file = widgets.HBox([filename_input, add_from_file_button])
updates_ui = widgets.HBox([update_all_button])
display(add_from_url_ui, add_from_url_file, updates_ui, save_button)

Loaded 304 games


Button(description='Save gamelist', style=ButtonStyle())

Checking for updates, 2024-12-21 22:02:43.806624
    Updating '23 Sisters' by Abere Lucifer
    Updating '53x – Homecoming' by Steam
    Updating 'A Day at the Spa' by ReallySlowGuy
    Updating 'A Family Venture' by WillTylor
    Updating 'A Few Days' by Patreon
    Updating 'A Foreign World' by Patreon
    Updating 'A Girl on a Train' by SubscribeStar
    Updating 'A House in the Rift' by Patreon
    Updating 'A Knight's Tale' by Patreon
    Updating 'A Moment of Bliss' by Itch.io
    Updating 'A Petal Among Thorns' by Lockheart
    Updating 'A Town Uncovered' by Patreon
    Updating 'A.O.A. Academy' by Patreon
    Updating 'AITA - Am I the Asshole' by Twisted Moon Games
    Updating 'Above the Clouds' by Mundo Games
    Updating 'Acting Lessons' by Patreon
    Updating 'Alchemist Trainer' by ShiroV8​
    Updating 'All the Wrong Things' by Goosenards
    Updating 'Anita's Discoveries' by Patreon
    Updating 'Apocalypse' by LEGACYUSER
    Updating 'Avalon' by Patreon
Error fetching o